In [5]:
# DB 접속
import os
import sys
import requests
import pymysql
import json
import pandas as pd
from sqlalchemy import create_engine

In [6]:
# air_pollution 테이블 결과 표시
url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty"
params = {'serviceKey':'SrBP4MJ4PYFoYhIxaHfCMHSC1yBK4gLjLleGwxw2QL0MA/Etyjf8JiX/2fhz5/sL4xrzckRXXjWNnS4agSqm5A==',
          'returnType':'json',
          'numOfRows':'100',
          'pageNo':'1',
          'sidoName':'서울'}
response = requests.get(url,params=params)

result = response.json()
df = pd.DataFrame(result['response']['body']['items'])
col = ['sidoName','so2Value','coValue','o3Value','no2Value','pm10Value']
df = df[col]

user = 'lim'
db = 'limdb'
password = 'Asd2004'
host = 'host.docker.internal' 
port = 3307
conn = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user,password,host,port,db)
engine = create_engine(conn)
df.to_sql(name='air_pollution', con=engine, if_exists='append', index=False)

query = 'SELECT* FROM air_pollution;'
pd.read_sql_query(query,engine)

,sidoName,so2Value,coValue,o3Value,no2Value,pm10Value
0,서울,0.002,0.3,0.088,0.012,41
1,서울,0.003,0.5,0.084,0.021,59
2,서울,0.003,0.3,0.088,0.009,43
3,서울,0.003,0.3,0.077,0.014,56
4,서울,0.004,0.4,0.079,0.017,43
...,...,...,...,...,...,...
195,서울,0.006,0.3,0.075,0.024,53
196,서울,0.003,0.3,0.111,0.013,57
197,서울,0.006,0.3,0.098,0.014,49
198,서울,0.004,0.4,0.092,0.013,60


In [7]:
# trend 테이블 결과 표시
header ={'X-Naver-Client-Id':'F0bMGPTVgCNbrYnJQLmq',
         'X-Naver-Client-Secret':'gs4shTAvr5',
         'Content-Type':'application/json'}
body = {'startDate':'2023-01-01',
        'endDate':'2023-12-31',
        'timeUnit':'date',
        'keywordGroups':[{'groupName':'벚꽃', 'keywords':['벚꽃','cherry blossom']},
                         {'groupName':'장범준', 'keywords':['장범준','버스커버스커']}
                         ],
        'device':'mo',
        'ages':['2','3','4']}
body = json.dumps(body)
url = 'https://openapi.naver.com/v1/datalab/search'

response = requests.post(url, headers=header, data=body)
result = json.loads(response.text)
user = 'lim'
db = 'limdb'
password = 'Asd2004'
host = 'host.docker.internal'
port = 3307
conn = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user,password,host,port,db)
engine = create_engine(conn)

for i in result['results']:
    df = pd.DataFrame(i['data'])
    df['title'] = i['title']
    df.to_sql('trend', con=engine, if_exists='append', index=False) 

query = "SELECT * FROM trend"
pd.read_sql_query(query, engine)

,period,ratio,title
0,2023-01-01,1.68570,벚꽃
1,2023-01-02,1.51634,벚꽃
2,2023-01-03,1.40212,벚꽃
3,2023-01-04,1.16581,벚꽃
4,2023-01-05,1.45726,벚꽃
...,...,...,...
725,2023-12-27,3.36746,장범준
726,2023-12-28,3.07995,장범준
727,2023-12-29,2.56400,장범준
728,2023-12-30,4.02126,장범준
